In [1]:
# These numbers come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
# DIMENSIONS specified because we are using the center of each voxel. They are saved 
import numpy as np

def get_voxel_centers_as_array(num_x_pts, num_y_pts, num_z_pts, increment, file):
    total_voxels = num_x_pts * num_y_pts * num_z_pts
    natee = 0
    next_print = total_voxels / 10
    
    voxel_centers = np.empty((num_x_pts, num_y_pts, num_z_pts), dtype=int)
    with open(file, 'r') as voxel_centers_file:
        voxel_centers_file.readline()
        for line in voxel_centers_file:
            tokens = line.split(',')
            x = int(float(tokens[0]))
            y = int(float(tokens[1]))
            z = int(float(tokens[2]))
            grain_id = int(tokens[3])
            voxel_centers[x // increment][y // increment][z // increment] = grain_id
            
            # Print progress
            natee += 1
            if natee >= next_print:
                print('Progress: ', natee / total_voxels)
                next_print += total_voxels / 10
    return voxel_centers

In [2]:
# This will give the nearest voxel center (x,y,z) based on the voxel size being the length of 1 side of the voxel
def find_nearest_voxel_center(x, y, z, voxel_size):
    voxel_x = int(((x + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_y = int(((y + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_z = int(((z + (voxel_size / 2)) // voxel_size) * voxel_size)
    return voxel_x, voxel_y, voxel_z

In [3]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with.
def get_crack_front_points(input_file):
    with open(input_file, 'r') as crack_points_file:
        crack_points_file.readline()
        crack_points = []
        for line in crack_points_file:
            tokens = line.split(',')
            crack_id = tokens[0]
            theta = tokens[1]
            x = tokens[5]
            y = tokens[6]
            z = tokens[7].strip()
            dadN3Dline = tokens[3]
            if int(crack_id) > 0:
                crack_points.append((crack_id, theta, x, y, z, dadN3Dline))
    return crack_points

In [4]:
def get_voxel_centers_index(x, y, z, voxel_size):
    return x // voxel_size, y // voxel_size, z // voxel_size

In [5]:
def get_xyz_from_voxel_centers_index(x, y, z, voxel_size):
    return x * voxel_size, y * voxel_size, z * voxel_size

In [6]:
def get_actual_grain_id(long_grain_id):
    return long_grain_id % 10000

In [48]:
def get_grain_ids_dictionary(voxel_centers):
    total = (len(voxel_centers)-1) * (len(voxel_centers[0])-1) *  (len(voxel_centers[0][0])-1)
    natee = 0
    next_print = total / 10
    grain_ids_dictionary = {}
    for z in range(len(voxel_centers[0][0])):
        for y in range(len(voxel_centers[0])):
            for x in range(len(voxel_centers)):
                grain_id = get_actual_grain_id(voxel_centers[x][y][z])
                if grain_id in grain_ids_dictionary:
                    grain_ids_dictionary[grain_id].append((x,y,z))
                else:
                    grain_ids_dictionary[grain_id] = [((x,y,z))]
                natee += 1
                if natee > next_print:
                    print(natee / total)
                    next_print += total / 10
    return grain_ids_dictionary

In [9]:
# Time: 2-3 minutes
voxel_centers = get_voxel_centers_as_array(201, 376, 301, 2, '../data/voxel-centers.csv')
print('done')

Progress:  0.10000001758367279
Progress:  0.20000003516734557
Progress:  0.3000000087918364
Progress:  0.4000000263755092
Progress:  0.5
Progress:  0.6000000175836728
Progress:  0.7000000351673455
Progress:  0.8000000087918364
Progress:  0.9000000263755091
done


In [10]:
crack_front_points = get_crack_front_points('../data/crack_front_growth_rates_1500ppcf_transformed.csv')

In [49]:
# Time: 2 minutes
grain_ids_dictionary = get_grain_ids_dictionary(voxel_centers)
print('done')

0.10000004444444445
0.20000004444444444
0.30000004444444445
0.4000000444444444
0.5000000444444445
0.6000000444444444
0.7000000444444444
0.8000000444444444
0.9000000444444445
1.0000000444444443
done


In [64]:
# TODO: Verify this works as best I can with tests/vis

voxel_size = 2
next_print = 3.15 / 100
crack_id_test = '1' # Use this to do each crack front iteratively
previously_looked_up = {}
block_size = 10

with open('../data/nearest_grain_boundary.csv', 'a') as file:
    if crack_id_test == '1':
        file.write('Crack ID, Theta, X, Y, Z, dadN3Dline, Nearest Grain ID, Grain Boundary X, Grain Boundary Y, Grain Boundary Z, Grain Boundary ID\n')
#     for crack_front_point in crack_front_points:
    for crack_front_point in [('1',1,block_size*2,block_size*2,block_size*2,.123)]:
        crack_id, theta, x, y, z, dadN3Dline = crack_front_point
        if crack_id_test == crack_id:
            x = float(x)
            y = float(y)
            z = float(z)

            # Get the nearest voxel_center for the x,y,z of the crack_front
            nearest_voxel_xyz = find_nearest_voxel_center(x, y, z, voxel_size)
            nearest_voxel_x, nearest_voxel_y, nearest_voxel_z = nearest_voxel_xyz

            # Get the indices and grain_id of the voxel we're looking at
            voxel_centers_index_x, voxel_centers_index_y, voxel_centers_index_z = \
                get_voxel_centers_index(nearest_voxel_x, nearest_voxel_y, nearest_voxel_z, voxel_size)
            nearest_grain_id = get_actual_grain_id(voxel_centers[voxel_centers_index_x][voxel_centers_index_y][voxel_centers_index_z])
            
            block_lower_x = max(0, voxel_centers_index_x-block_size)
            block_upper_x = min(len(voxel_centers)-1, voxel_centers_index_x+block_size)+1
            block_lower_y = max(0, voxel_centers_index_y-block_size)
            block_upper_y = min(len(voxel_centers[0])-1, voxel_centers_index_y+block_size)+1
            block_lower_z = max(0, voxel_centers_index_z-block_size)
            block_upper_z = min(len(voxel_centers[0][0])-1, voxel_centers_index_z+block_size)+1
            block = voxel_centers[block_lower_x:block_upper_x, block_lower_y:block_upper_y, block_lower_z:block_upper_z]
            
            to_search = []
            pts_of_grain_id = grain_ids_dictionary[nearest_grain_id]
            for block_z in range(len(block[0][0])):
                for block_y in range(len(block[0])):
                    for block_x in range(len(block)):
                        test = (block_x, block_y, block_z)
                        if test not in pts_of_grain_id:
                            to_search.append(test)
            
            min_distance = 999999999
            min_x = -999999999
            min_y = -999999999
            min_z = -999999999
            for pt in to_search:
                pt_x_index = pt[0]
                pt_y_index = pt[1]
                pt_z_index = pt[2]
                # Got back to x,y,z from indices
                pt_x, pt_y, pt_z = get_xyz_from_voxel_centers_index(pt_x_index, pt_y_index, pt_z_index, voxel_size)
                nearest_voxel_x, nearest_voxel_y, nearest_voxel_z
                distance = ((pt_x-nearest_voxel_x)**2 + (pt_y-nearest_voxel_y)**2 + (pt_z-nearest_voxel_z)**2)**.5
                if distance < min_distance:
                    min_distance = distance
                    min_x = pt_x
                    min_y = pt_y
                    min_z = pt_z
            
            if min_distance == 999999999:
                raise RuntimeError('No grain boundary found with block_size: ' + str(block_size))
            print(min_distance, min_x, min_y, min_z)
#             # Find the x,y,z,grain_id of the nearest grain boundary
#             if nearest_voxel_xyz in previously_looked_up:
#                 grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id = previously_looked_up[nearest_voxel_xyz]
#             else:
                
#                 previously_looked_up[nearest_voxel_xyz] = (grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id)

            # Got back to x,y,z from indices
#             grain_boundary_x, grain_boundary_y, grain_boundary_z = get_xyz_from_voxel_centers_index(grain_boundary_index_x, \
#                                                             grain_boundary_index_y, grain_boundary_index_z, voxel_size)

#             temp = str(crack_id)+','+str(theta)+','+str(x)+','+str(y)+','+str(z)+','+str(dadN3Dline)+','+str(nearest_grain_id)+','+\
#                 str(grain_boundary_x)+','+str(grain_boundary_y)+','+str(grain_boundary_z)+','+str(grain_boundary_id)+'\n'
#             file.write(temp)

#             if float(theta) > next_print:
#                 print('Progress: ', float(theta) / 3.15)
#                 next_print += 3.15 / 100

print('done')

6.0 20 14 20
done


[(56, 275, 29),
 (53, 274, 30),
 (54, 272, 31),
 (55, 272, 31),
 (56, 272, 31),
 (57, 272, 31),
 (54, 273, 31),
 (55, 273, 31),
 (56, 273, 31),
 (57, 273, 31),
 (58, 273, 31),
 (59, 273, 31),
 (52, 274, 31),
 (53, 274, 31),
 (54, 274, 31),
 (55, 274, 31),
 (56, 274, 31),
 (57, 274, 31),
 (58, 274, 31),
 (59, 274, 31),
 (60, 274, 31),
 (52, 275, 31),
 (53, 275, 31),
 (54, 275, 31),
 (55, 275, 31),
 (56, 275, 31),
 (57, 275, 31),
 (58, 275, 31),
 (59, 275, 31),
 (54, 276, 31),
 (55, 276, 31),
 (56, 276, 31),
 (57, 276, 31),
 (58, 276, 31),
 (54, 277, 31),
 (55, 277, 31),
 (56, 277, 31),
 (57, 277, 31),
 (58, 277, 31),
 (55, 278, 31),
 (56, 278, 31),
 (57, 278, 31),
 (56, 279, 31),
 (53, 271, 32),
 (54, 271, 32),
 (53, 272, 32),
 (54, 272, 32),
 (55, 272, 32),
 (56, 272, 32),
 (57, 272, 32),
 (52, 273, 32),
 (54, 273, 32),
 (55, 273, 32),
 (56, 273, 32),
 (57, 273, 32),
 (58, 273, 32),
 (59, 273, 32),
 (52, 274, 32),
 (53, 274, 32),
 (54, 274, 32),
 (55, 274, 32),
 (56, 274, 32),
 (57, 27

In [6]:
# Given an array of the voxel centers arr[x][y][z] = grain_id at those 'coordinates / voxel_size'. Returns the nearest 
# voxel center that has a different grain_id associated with it. Basically, just does a breadth-first search...
from collections import deque

def find_nearest_grain_boundary(voxel_centers, checked_voxels, x, y, z, grain_id):
    grain_id = get_actual_grain_id(grain_id)
    queue = deque([(x,y,z)])
    
    while len(queue) > 0:
        u = queue.popleft()
        current_x, current_y, current_z = u
        
        # voxel_centers[current_x][current_y][current_z] is the grain_id of the voxel we're checking
        current_grain_id = get_actual_grain_id(voxel_centers[current_x][current_y][current_z])
        if current_grain_id != grain_id:
            # TODO: If it's in front of the crack surface then return, otherwise, continue
            return (current_x, current_y, current_z, current_grain_id)
        checked_voxels[current_x][current_y][current_z] = True
        
        if current_x < len(voxel_centers) - 1 and not checked_voxels[current_x + 1][current_y][current_z]:
            queue.append((current_x + 1, current_y, current_z))
            
        if current_x > 0 and not checked_voxels[current_x - 1][current_y][current_z]:
            queue.append((current_x - 1, current_y, current_z))
            
        if current_y < len(voxel_centers[0]) - 1 and not checked_voxels[current_x][current_y + 1][current_z]:
            queue.append((current_x, current_y + 1, current_z))
            
        if current_y > 0 and not checked_voxels[current_x][current_y - 1][current_z]:
            queue.append((current_x, current_y - 1, current_z))
            
        if current_z < len(voxel_centers[0][0]) - 1 and not checked_voxels[current_x][current_y][current_z + 1]:
            queue.append((current_x, current_y, current_z + 1))
            
        if current_z > 0 and not checked_voxels[current_x][current_y][current_z - 1]:
            queue.append((current_x, current_y, current_z - 1))
    
    raise RuntimeError('Grain boundary not found')